<h1>Segmenting and Clustering Neighborhoods in Toronto<h1>

<h2>Question 1<h2>
<p><i> Web Scrapping <p><i>
<p><i> Data Cleaning <p><i>
<p><i> Dataframe creation <p><i>


In [2]:
#Import libraries needed
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [3]:
#Wikipedia url with the information from Toronto's neighborhoods
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
#Capture the hmtl data from the url
data = requests.get(url).text

In [5]:
#Create BeautifulSoup object with the data captured
soup = BeautifulSoup(data,'html5lib')

In [6]:
#Find the target table and store it in a variable
table = soup.find('table')

In [7]:
"""
Store the table contents in a list by doing the following:
1. Ignore postal codes that do not have a neighborhood assigned
2. The first three letters in each cell content correspond to the postal code
3. The Borough corresponds to the text outside of the paranthesis 
4. Clean the data inside of the paranthesis to get the neighborhoods

"""
table_contents = []
for row in table.find_all('td'):
  cell = {}
  if row.span.text == 'Not assigned':
    pass
  else:
    cell['PostalCode'] = row.p.text[:3]
    cell['Borough'] = (row.span.text).split('(')[0]
    cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
    table_contents.append(cell)

In [8]:
print(table_contents)

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'PostalCode': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'PostalCode': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park, Harbourfront'}, {'PostalCode': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor, Lawrence Heights'}, {'PostalCode': 'M7A', 'Borough': "Queen's Park", 'Neighborhood': 'Ontario Provincial Government'}, {'PostalCode': 'M9A', 'Borough': 'Etobicoke', 'Neighborhood': 'Islington Avenue'}, {'PostalCode': 'M1B', 'Borough': 'Scarborough', 'Neighborhood': 'Malvern, Rouge'}, {'PostalCode': 'M3B', 'Borough': 'North York', 'Neighborhood': 'Don Mills North'}, {'PostalCode': 'M4B', 'Borough': 'East York', 'Neighborhood': 'Parkview Hill, Woodbine Gardens'}, {'PostalCode': 'M5B', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Garden District, Ryerson'}, {'PostalCode': 'M6B', 'Borough': 'North York', 'Neighborhood': 'Glencairn'}, {'PostalCode': 'M9

In [9]:
#Store the list information in a pandas dataframe
df = pd.DataFrame(table_contents)

In [10]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [11]:
#Clean the data in the dataframe by modifying the data that has additional information which is not needed
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [13]:
df.shape

(103, 3)

<hr size="5" width="100%" color="black">  
<h2>Question 2<h2>
<p><i> Determine Longitude and Latitude of Neighborhoods <p><i>


In [20]:
!pip install geocoder
!pip install geopy
import geocoder
from geopy.geocoders import Nominatim

     |████████████████████████████████| 102kB 3.6MB/s 


In [ ]:
#Two method were used to try and determine the latitude and logitude of the neighborhoods
#Both methods kept returning None for various locations so the Geospatial_Coordinates.csv provided by Coursera was used in order to continue with the exercise
"""
loc_list = {}
for index in df.index:
  print(df['PostalCode'][index])
  locator = Nominatim(user_agent="myGeocoder")
  location = locator.geocode('{}, Toronto, Ontario'.format(df['PostalCode'][index]))
  
  loc_list['Latitude'] = location.latitude
  loc_list['Longitude']  = location.longitude
"""

"""
loc_list = {}
for index in df.index:
  lat_lng_coords = None

  while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(df['PostalCode'][index]))
    lat_lng_coords = g.latlng
  
  loc_list['Latitude'] = lat_lng_coords[0]
  loc_list['Longitude']  = lat_lng_coords[1]

"""


In [59]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [61]:
df_geo = pd.read_csv("/content/drive/MyDrive/Geospatial_Coordinates.csv")

In [65]:
df_geo.rename(columns={'Postal Code' : 'PostalCode'}, inplace=True)
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [66]:
df_merge = pd.merge(df, df_geo, on='PostalCode')

In [67]:
df_merge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


<hr size="5" width="100%" color="black">  
<h2>Question 3<h2>
<p><i> Determine Longitude and Latitude of Neighborhoods <p><i>

In [68]:
import json 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="mygeocoder")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [71]:
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lng, borough, neighborhood in zip(df_merge['Latitude'],df_merge['Longitude'],df_merge['Borough'], df_merge['Neighborhood']):
  label = '{}, {}'.format(neighborhood,borough)
  label = folium.Popup(label,parse_html=True)
  folium.CircleMarker(
      [lat,lng],
      radius=5,
      popup=label,
      color='purple',
      fill=True,
      fill_color='#800080',
      fill_opacity=0.7,
      parse_html=False).add_to(map_toronto)
map_toronto

In [72]:
toronto_data = df_merge[df_merge['Borough'].str.contains('Toronto',na=False)].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [73]:
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lng, borough, neighborhood in zip(toronto_data['Latitude'],toronto_data['Longitude'],toronto_data['Borough'], toronto_data['Neighborhood']):
  label = '{}, {}'.format(neighborhood,borough)
  label = folium.Popup(label,parse_html=True)
  folium.CircleMarker(
      [lat,lng],
      radius=5,
      popup=label,
      color='purple',
      fill=True,
      fill_color='#800080',
      fill_opacity=0.7,
      parse_html=False).add_to(map_toronto)
map_toronto